In [ ]:
# pip3 install googletrans==3.1.0a0

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5TokenizerFast, T5ForConditionalGeneration, EncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## **Traductor**

In [2]:
trans = Translator()
print(trans.translate('Pues hazme los deberes, furcia barata!').text)

Well, do my homework for me, you cheap whore!


## **Dataset**

In [19]:
csv = pd.read_csv('2_detox_dataset_trans.csv')
csv = pd.read_csv('english20k_dataset.csv')
df = pd.DataFrame(csv)

train_dataframe, val_dataframe = train_test_split(df, train_size=0.75, random_state=42) # random_state=42 for reproducibility

train_toxic_texts = list(train_dataframe['toxic_sentence'])
train_neutral_texts = list(train_dataframe['neutral_sentence'])

val_toxic_texts = list(val_dataframe['toxic_sentence'])
val_neutral_texts = list(val_dataframe['neutral_sentence'])

## **Detoxificador**

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
toxic_encodings_train = tokenizer(train_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_train = tokenizer(train_neutral_texts, truncation=True, padding=True, return_tensors="pt")

toxic_encodings_val = tokenizer(val_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_val = tokenizer(val_neutral_texts, truncation=True, padding=True, return_tensors="pt")

In [ ]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

train_dataset = DetoxDataset(toxic_encodings_train, neutral_encodings_train)
val_dataset = DetoxDataset(toxic_encodings_val, neutral_encodings_val)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    num_train_epochs=5,
    report_to=["none"], # para no pedir login de 'wandb' y otros
    fp16=True, # acelerar entrenaminento 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()

  0%|          | 0/609 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 0.3892, 'eval_samples_per_second': 925.039, 'eval_steps_per_second': 30.835, 'epoch': 1.0}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 0.3588, 'eval_samples_per_second': 1003.317, 'eval_steps_per_second': 33.444, 'epoch': 2.0}
{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 2.46}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 0.361, 'eval_samples_per_second': 997.165, 'eval_steps_per_second': 33.239, 'epoch': 3.0}
{'train_runtime': 100.5674, 'train_samples_per_second': 96.652, 'train_steps_per_second': 6.056, 'train_loss': 0.0, 'epoch': 3.0}


TrainOutput(global_step=609, training_loss=0.0, metrics={'train_runtime': 100.5674, 'train_samples_per_second': 96.652, 'train_steps_per_second': 6.056, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
model.save_pretrained("flan-t5-detoxificado")
tokenizer.save_pretrained("flan-t5-detoxificado")

## **PRUEBAS DETOX**

In [11]:
# texto_prueba = "Well, do my homework for me, you cheap whore!" 
texto_prueba = "shut your angry ass up" 

# Tokenizar la oración tóxica
inputs = tokenizer(texto_prueba, return_tensors="pt", truncation=True, padding=True)
# Mover los tensores a la GPU, si está disponible
inputs = {key: val.to(device) for key, val in inputs.items()}

outputs = model.generate(inputs["input_ids"])
texto_neutralizado = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Detoxed text: {texto_neutralizado}")

Detoxed text: shut your angry ass up


In [9]:
def detoxify_sentence(text: str):
    # Tokenizar la oración tóxica
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Mover los tensores a la GPU, si está disponible
    inputs = {key: val.to(device) for key, val in inputs.items()}

    outputs = model.generate(inputs["input_ids"])
    texto_neutralizado = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_neutralizado

In [ ]:
test_sentences = [
    "shut your angry ass up",
    "You are a stupid asshole."
]

for sentence in test_sentences:
    neutral_sentence = detoxify_sentence(sentence)
    print(f"Original: {sentence} -> Neutral: {neutral_sentence}") 

Original: shut your angry ass up -> Neutral: shut your angry ass up
Original: You are a stupid asshole. -> Neutral: You are stupid asshole.


---

---

## **BUCLE DE TRADUCCIÓN** (26 mins)

In [ ]:
csv = pd.read_csv('detox_dataset.csv')
df = pd.DataFrame(csv)

trans = Translator()
# print(trans.translate(df.iloc[400]['toxic_sentence'], dest='en').text)
print(trans.translate("ታምራት ነገራ ፅንፈኞችን ከላይ እስከታች ሰብስበህ ሁለት ሰንበት ፀበልና ስልጠና ስጥልን", dest='en').text)

In [ ]:
csv_trans = pd.read_csv('detox_dataset_trans.csv')
df_trans = pd.DataFrame(csv_trans)

In [ ]:
for index in range(len(df)):
    traductor = Translator()
    toxic_translated = traductor.translate(df.iloc[index]['toxic_sentence'], dest='en').text
    neutral_translated = traductor.translate(df.iloc[index]['neutral_sentence'], dest='en').text
    new_row = [toxic_translated, neutral_translated]
    df_trans.loc[len(df_trans)] = new_row
    # break

df_trans.to_csv('detox_dataset_trans.csv', index=False)